# API tests

In [8]:
import os
import json
import twitter_functions as tf # custom utility functions

from importlib import reload

Some helpful articles and documentation
* [Search query builder to validate queries](https://twitter-query-builder.glitch.me/)
* [List of operators here](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#availability) 
* [Keep an eye open for the twarc utility](https://github.com/DocNow/twarc/wiki/twarc2)
* [Surhem Parack's office hours](https://twittercommunity.com/t/announcing-monthly-office-hours-for-the-academic-research-product-track/151028) 
* [Twitter API v2 sample codes](https://github.com/twitterdev/Twitter-API-v2-sample-code)
* [My project dashboard](https://developer.twitter.com/en/portal/dashboard)
* [Tools for Twitter project](https://github.com/texifter/tools-for-twitter/blob/main/run_twitter_search_all.py)

In [11]:
# read the api key
API_info = {}
with open("twitter_API.txt") as f:
    for line in f:
        key, val = line.partition("=")[::2]
        if key == 'bearer_token':
            API_info[key] = val.strip('\n')
            
bearer_token = API_info['bearer_token']

## Tutorial example

In [12]:
with open('configs/test_config.json') as json_file:
    params = json.load(json_file)

In [22]:
reload(tf)

<module 'twitter_functions' from '/home/jana/Projects/CSS_twitter_inoculation/analysis/code/twitter_functions.py'>

In [18]:
config = {
  "bearer_token": bearer_token,
  "params": params,
  "write_path": "../resources/tweets.txt"
}

tf.validate_config(config)

In [23]:

count = 0
next_token = None

if 'params' in config and 'max_results' in config['params']:
    max_results = config['params']['max_results']
else:
    max_results = 500

while count < max_results:

    json_response = tf.search_tweets(config, next_token)
    tweets = tf.get_formatted_tweets(json_response)
    tf.write_to_file(config['write_path'], tweets)
    result_count = json_response['meta']['result_count']
    count += result_count

    if 'meta' in json_response and 'next_token' in json_response['meta']:
        next_token = json_response['meta']['next_token']
    else:
        break

In [24]:
tweets[0]

{'text': 'RT @monicaoltra: Voleu saber una bona notícia⁉️\n\n➡️”En dos hospitales valencianos no hay enfermos covid y en ocho ninguno en la UCI”\n\nI la…',
 'context_annotations': [{'domain': {'id': '123',
    'name': 'Ongoing News Story',
    'description': "Ongoing News Stories like 'Brexit'"},
   'entity': {'id': '1220701888179359745', 'name': 'COVID-19'}}],
 'author_id': '1380038028484345857',
 'created_at': '2021-05-11T23:59:59.000Z',
 'id': '1392268281181089799',
 'user': {'username': 'targaryen88',
  'name': 'Targaryen88',
  'id': '1380038028484345857',
  'created_at': '2021-04-08T06:01:40.000Z'}}

## Full archival path endpoint

In [7]:
# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
#bearer_token = os.environ.get("BEARER_TOKEN")
bearer_token = API_info['bearer_token']

# full archival path endpoint
search_url = "https://api.twitter.com/2/tweets/search/all"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev',
                'tweet.fields': 'author_id'}


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


headers = create_headers(bearer_token)
json_response = connect_to_endpoint(search_url, headers, query_params)
print(json.dumps(json_response, indent=4, sort_keys=True))


200
{
    "data": [
        {
            "author_id": "14104570",
            "id": "1393194226452684809",
            "text": "Hey @jack from @twitter. Could you all add spaces\u2019 voice chat to direct messages? \n\n@instagram has the direct call feature already.\n\nEveryone wins with competition.\n\nI bet someone at the @TwitterDev could make this happen by the end of the day. \n\n#TwitterDev #TwitterDevFlex"
        },
        {
            "author_id": "2244994945",
            "id": "1392887366507970561",
            "text": "If you use R code to do research with the #TwitterAPI, you won't want to miss this.\n\nWe\u2019re hosting @cbarrie and @justin_ct_ho who built the academictwitteR package: https://t.co/Of3i08APR4 #rstats \n\nTune in next Thursday the 20th at 10am PT on https://t.co/GrtBOXyHmB. \ud83d\udc40"
        },
        {
            "author_id": "73916222",
            "id": "1392492479732199430",
            "text": "@ReallySwara There's no dislike button ....\n#ja